In [ ]:
from surprise import SVD, SVDpp, NMF, Dataset, Reader
from surprise.model_selection import cross_validate, GridSearchCV
import pandas as pd
import numpy as np

In [ ]:
ratings_url = "https://files.grouplens.org/datasets/movielens/ml-latest-small/ratings.csv"
movies_url = "https://files.grouplens.org/datasets/movielens/ml-latest-small/movies.csv"

ratings = pd.read_csv(ratings_url)
movies = pd.read_csv(movies_url)

data = pd.merge(ratings, movies, on='movieId')
print(data.head())

reader = Reader(rating_scale=(0.5, 5.0))
dataset = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

HTTPError: HTTP Error 404: Not Found

In [1]:
param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.1]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1)
gs.fit(dataset)

print("Best RMSE score:", gs.best_score['rmse'])
print("Best parameters:", gs.best_params['rmse'])

best_svd = gs.best_estimator['rmse']
cross_validate(best_svd, dataset, measures=['RMSE', 'MAE'], cv=3, verbose=True)

NameError: name 'GridSearchCV' is not defined

In [2]:
svdpp = SVDpp()
cross_validate(svdpp, dataset, measures=['RMSE', 'MAE'], cv=3, verbose=True)

nmf = NMF()
cross_validate(nmf, dataset, measures=['RMSE', 'MAE'], cv=3, verbose=True)

NameError: name 'SVDpp' is not defined

In [4]:
def initialize_matrix(n_users, n_items, n_factors):
    np.random.seed(42)
    P = np.random.rand(n_users, n_factors)
    Q = np.random.rand(n_items, n_factors)
    return P, Q

In [5]:
def loss_function(R, P, Q, lambda_reg):
    loss = 0
    for u, i, r_ui in R:
        prediction = np.dot(P[u, :], Q[i, :])
        loss += (r_ui - prediction) ** 2
    loss += lambda_reg * (np.sum(np.square(P)) + np.sum(np.square(Q)))
    return loss

In [6]:
def gradient_descent(R, P, Q, lambda_reg, lr, epochs):
    for epoch in range(epochs):
        for u, i, r_ui in R:
            error = r_ui - np.dot(P[u, :], Q[i, :])
            P[u, :] += lr * (error * Q[i, :] - lambda_reg * P[u, :])
            Q[i, :] += lr * (error * P[u, :] - lambda_reg * Q[i, :])

        loss = loss_function(R, P, Q, lambda_reg)
        print(f"Epoch {epoch + 1}, Loss: {loss}")
    return P, Q

In [9]:
n_users = ratings['userId'].nunique()
n_items = ratings['movieId'].nunique()
R = [(int(row['userId']) - 1, int(row['movieId']) - 1, row['rating']) for _, row in ratings.iterrows()]

n_factors = 50
lambda_reg = 0.02
lr = 0.005
epochs = 20

P, Q = initialize_matrix(n_users, n_items, n_factors)
P, Q = gradient_descent(R, P, Q, lambda_reg, lr, epochs)

NameError: name 'ratings' is not defined